In [ ]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
# 로드될 때까지 기다리는 설정을 위한 import
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# 상품 관련 내용을 크롤링하기 위해 import
from bs4 import BeautifulSoup
# 상품의 정보를 예쁘게 출력하기 위해 import
import json
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
# 엑셀 저장을 위해 추가
import pandas as pd
# 엑셀에 날짜 저장을 위해 import
from datetime import datetime

# 옵션 설정: Headless 모드로 실행되도록 설정 -> 창이 켜지지 않고 vscode 내에서 바로바로 실행 됨 250513 제리
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저 창 안 띄움
options.add_argument('--disable-gpu')  # GPU 가속 비활성화 (Windows 환경에서 필요할 수 있음)
options.add_argument('--no-sandbox')  # 리눅스에서 root 권한 문제 해결용
options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 제한 문제 해결용

# 크롬 드라이버 실행 (옵션 추가)
driver = webdriver.Chrome(options=options)

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://youtu.be/w7uwgwj6aZM?si=kMxeRqkIo6AD--pn') # 침착맨 "버거킹, 우리의 아이디어를 꼭 들어주길 바랍니다"
# 유튜브 영상의 제목 html 요소가 로드될 때까지 최대 60초 대기(xpath 방식으로 구현)
base_info_results = {}

"""
영상 제목, 채널명, 구독자 수, 조회수, 업로드 날짜
"""
# XPath는 바뀔 수도 있으니, 에러가 뜬다면 html 다시 뜯으러 가야 함.
base_info = {
    "title": '//*[@id="title"]/h1',
    "channel": '//*[@id="text"]/a',
    "subscriber": '//*[@id="owner-sub-count"]',
    "views": '//*[@id="info"]/span[1]',
    "upload_date": '//*[@id="info"]/span[3]',
}

# base_info에 담아둔 key에 따른 value들을 xpath를 통해 돌아가면서 들고옴 (빈복 작업)
for bases, xpath in base_info.items():
    try:
        # 로딩될 때까지 최대 60초 기다림
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        # bases에 들어간 title, channel 등등의 내용(text)을 "base_info_results" 딕셔너리에 추가
        base_info_results[bases] = element.text
    # error 처리
    except Exception as e:
        print(f"Error fetching {bases}: {e}")
        base_info_results[bases] = "Not found"


# 정보 출력
print("\n=== Base Info ===")
for key, value in base_info_results.items():
    print(f"{key}: {value}")

# 오늘 날짜를 문자열로 저장 (형식: YYMMDD)
today_str = datetime.today().strftime('%y%m%d')

# 파일명 생성
filename = f'youtube_video_{today_str}.xlsx'

# DataFrame을 엑셀로 저장
df = pd.DataFrame([base_info_results])
df.to_excel(filename, index=False)

print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

# 드라이버 종료
try:
    # scraping logic here
    pass
finally:
    driver.quit()


=== Base Info ===
title: 버거킹, 우리의 아이디어를 꼭 들어주길 바랍니다
channel: 침착맨
subscriber: 구독자 281만명
views: 조회수 353,248회
upload_date: 22시간 전
